In [ ]:
pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.config('spark.diver.host', 'localhost').appName('localhost').appName('filter_search').getOrCreate()

In [ ]:
fifa = spark.read.csv('fifa19.csv', header = True, inferSchema=True)
fifa.printSchema()

root
 |-- no: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nul

### select와 orderBy

In [ ]:
# SELECT는 특정column만 가져올 수 있음 (DataFrame 반환)
fifa1 = fifa.select('Name', 'Age', 'Club')
print(type(fifa1))
fifa1.show(3)

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------------+---+-------------------+
|             Name|Age|               Club|
+-----------------+---+-------------------+
|         L. Messi| 31|       FC Barcelona|
|Cristiano Ronaldo| 33|           Juventus|
|        Neymar Jr| 26|Paris Saint-Germain|
+-----------------+---+-------------------+
only showing top 3 rows



In [ ]:
# oderBy는 지정된 column 기준으로 정렬(오름차순)할 수 있음
fifa2 = fifa1.orderBy('Age')
fifa2.show(3)
# column명은 'Age', fifa1['Age'], fifa1.Age형태로 사용할 수 있음

+-----------+---+-----------+
|       Name|Age|       Club|
+-----------+---+-----------+
|  Y. Roemer| 16|  VVV-Venlo|
|W. Geubbels| 16|  AS Monaco|
|J. Italiano| 16|Perth Glory|
+-----------+---+-----------+
only showing top 3 rows



In [ ]:
# 내림차순 정렬 column.desc()
fifa3 = fifa1.orderBy(fifa1['Age'].desc())
fifa3.show(3)
# desc()를 설정할 때, column명만 사용하면 에러 발생
# 반드시 DataFream['column'] 또는 DataFrame.column 을 사용해야 함
# DataFrame.column은 column명에 공백이 있는 경우 오류 발생

+-------------+---+------------------+
|         Name|Age|              Club|
+-------------+---+------------------+
|     O. Pérez| 45|           Pachuca|
|K. Pilkington| 44|  Cambridge United|
|    T. Warner| 44|Accrington Stanley|
+-------------+---+------------------+
only showing top 3 rows



### where

In [ ]:
# where는 조건식에 대해 참인 행만 반환
fifa4 = fifa.where(fifa.Age > 40).select('Name','Age', 'Nationality')
fifa4.show(5)

+-----------+---+-----------+
|       Name|Age|Nationality|
+-----------+---+-----------+
|  J. Villar| 41|   Paraguay|
|   B. Nivet| 41|     France|
|   O. Pérez| 45|     Mexico|
|   C. Muñoz| 41|  Argentina|
|S. Narazaki| 42|      Japan|
+-----------+---+-----------+
only showing top 5 rows



In [ ]:
# isin: 지정된 리스트에 포함된 값일 경우 True
# startswith : 지정된 문자로 시작하는 경우 True
# endswtih : 지정된 문자로 끝나는 경우 True
# like : 지정된 문자열을 포함한값일 경우 True

In [ ]:
# Club이 토트넘, 바르셀로나인 행의 None과 Club column만 반환
fifa.select('Name', 'Club').where(fifa.Club.isin('Tottenham Hotspur', 'FC Barcelona')).show()

+---------------+-----------------+
|           Name|             Club|
+---------------+-----------------+
|       L. Messi|     FC Barcelona|
|      L. Suárez|     FC Barcelona|
|        H. Kane|Tottenham Hotspur|
|  M. ter Stegen|     FC Barcelona|
|Sergio Busquets|     FC Barcelona|
|     C. Eriksen|Tottenham Hotspur|
|       Coutinho|     FC Barcelona|
|      H. Lloris|Tottenham Hotspur|
|      S. Umtiti|     FC Barcelona|
|     Jordi Alba|     FC Barcelona|
|  J. Vertonghen|Tottenham Hotspur|
|     I. Rakitić|     FC Barcelona|
|          Piqué|     FC Barcelona|
|T. Alderweireld|Tottenham Hotspur|
|       A. Vidal|     FC Barcelona|
|     D. Sánchez|Tottenham Hotspur|
|        D. Alli|Tottenham Hotspur|
|         H. Son|Tottenham Hotspur|
|     O. Dembélé|     FC Barcelona|
|    Lucas Moura|Tottenham Hotspur|
+---------------+-----------------+
only showing top 20 rows



In [ ]:
# Name column의 문자열이 L로 시작하는 선수의 Name과 Overall
# Name column의 문자열이 Lee로 끝나는 선수의 Name과 Overall
fifa.select('Name', 'Overall').where(fifa.Name.startswith('L')).show(5)
fifa.select('Name', 'Overall').where(fifa.Name.endswith('Lee')).show(5)

+----------+-------+
|      Name|Overall|
+----------+-------+
|  L. Messi|     94|
| L. Modrić|     91|
| L. Suárez|     91|
|L. Insigne|     88|
|   L. Sané|     86|
+----------+-------+
only showing top 5 rows

+----------+-------+
|      Name|Overall|
+----------+-------+
|    K. Lee|     71|
|Kangin Lee|     70|
|    O. Lee|     68|
|    T. Lee|     64|
|    E. Lee|     63|
+----------+-------+



In [ ]:
# Club column의 값이 Tottenham을 포함하는 행만 반환
# %위치에는 다른 문자/문자열이 있을 수 있다는 의미
fifa.select('Name', 'Age', 'Club').where(fifa.Club.like('%Tottenham%')).show(5)

+---------------+---+-----------------+
|           Name|Age|             Club|
+---------------+---+-----------------+
|        H. Kane| 24|Tottenham Hotspur|
|     C. Eriksen| 26|Tottenham Hotspur|
|      H. Lloris| 31|Tottenham Hotspur|
|  J. Vertonghen| 31|Tottenham Hotspur|
|T. Alderweireld| 29|Tottenham Hotspur|
+---------------+---+-----------------+
only showing top 5 rows



In [ ]:
### 과제) 나이가 38세 이상인 선수의 Nationality, Name, Overall, Age를 Overall기준으로 내림차순 정려 후 상위 10개 행을 보이시오.

In [ ]:
fifa6 = fifa.where(fifa.Age >= 38).select('Name', 'Overall', 'Age', 'Nationality')
fifa6 = fifa6.orderBy(fifa['Overall'].desc())
fifa6.show(10)

+-------------+-------+---+--------------+
|         Name|Overall|Age|   Nationality|
+-------------+-------+---+--------------+
|    G. Buffon|     88| 40|         Italy|
|S. Sorrentino|     80| 39|         Italy|
|       Hilton|     78| 40|        Brazil|
|    J. Villar|     77| 41|      Paraguay|
|  A. Bizzarri|     76| 40|     Argentina|
|   P. Guiñazú|     75| 39|     Argentina|
|   C. Pizarro|     74| 39|          Peru|
|     A. Boruc|     74| 38|        Poland|
|    J. Drobný|     73| 38|Czech Republic|
|    Cifuentes|     73| 39|         Spain|
+-------------+-------+---+--------------+
only showing top 10 rows

